# EDA — Projeto Previsão de Ativos

## Este notebook realiza a análise exploratória dos dados **Gold** oriundos da ETL:

1. Análise exploratória dos preços: volatilidade, retornos, distribuições, anomalias, comportamento cross-sectional.
2. Avaliação de consistência e completude do calendário: dias faltantes, alinhamento entre ativos, impacto em janelas móveis.
3. Exploração das features Gold: retornos, EMAs, lags, volatilidades, ratios, indicadores de tendência.
4. Exploração do benchmark: correlação ativo-IBOV, betas implícitos, sensibilidade.
5. Estudo da variável alvo: balanceamento, proporção de subidas/quedas, autocorrelação.

Observação: este notebook assume que os dados Gold já vêm limpos e consolidados pela etapa de ETL.

In [23]:
import duckdb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from pathlib import Path

from src.auxiliares import read_parquet_robust

In [24]:
# Configurações de exibição
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)

## 0. Configurações gerais

In [28]:
# %% Conexão com o warehouse DuckDB e leitura da tabela Gold principal

PROJECT_ROOT = Path("/home/rusch/Área de trabalho/Projeto_ativos/template")     
DATA_DIR     = PROJECT_ROOT / "data"
DB_PATH      = DATA_DIR / "warehouse.duckdb"

if not DB_PATH.exists():
    raise FileNotFoundError(f"warehouse.duckdb não encontrado em {DB_PATH}")

con = duckdb.connect(DB_PATH.as_posix())

In [29]:
# Lê a tabela principal de features diárias
df = con.execute("SELECT * FROM asset_features_daily").df()

In [30]:
# (Opcional) Lê também a tabela de KPIs para uso futuro
df_kpis = con.execute("SELECT * FROM asset_kpis_summary").df()

In [31]:
con.close()

In [32]:
print("Dimensões asset_features_daily:", df.shape)
print("Colunas asset_features_daily:")
print(df.columns.tolist())

Dimensões asset_features_daily: (46969, 25)
Colunas asset_features_daily:
['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'ret_1d', 'ret_5d', 'vol_21d', 'ema_9', 'ema_72', 'ema_200', 'ema_9_72_ratio', 'ema_9_200_ratio', 'ret_1d_lag1', 'ret_1d_lag2', 'ret_1d_lag3', 'futuro_ret_1d', 'target_direction', 'ibov_close', 'ibov_ret_1d', 'ibov_ret_lag1', 'ibov_ret_lag2', 'ibov_ret_lag3']


In [35]:
# %% Configuração das colunas principais (ajuste conforme o schema real)

# Aqui você ajusta **depois de olhar o print de df.columns**
DATE_COL   = "date"        # coluna de data (ex.: "date", "ref_date", etc.)
TICKER_COL = "ticker"      # coluna do ativo (ex.: "ticker", "asset", "symbol")
PRICE_COL  = "close"       # preço de fechamento (ex.: "close", "adj_close")
VOLUME_COL = "volume"      # se não existir, pode deixar como None ou ignorar depois
TARGET_COL = "target_up"   # nome da coluna alvo (subida/queda), se já existir na tabela

# Benchmark (ajuste se o IBOV vier como ticker ou como coluna separada)
BENCHMARK_TICKER     = "IBOV"          # se o IBOV estiver junto com os demais ativos
BENCHMARK_PRICE_COL  = "ibov_close"    # se houver uma coluna específica de preço do IBOV

# Lags que vamos usar na ACF do alvo
TARGET_ACF_LAGS = [1, 2, 3, 5]

# Conversão de data e ordenação
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values([TICKER_COL, DATE_COL]).reset_index(drop=True)

In [37]:
df_kpis.head()

,ticker,mean_ret_1d,vol_daily,skew_ret,kurtosis_ret,vol_annual,sharpe_like,hit_ratio,max_drawdown
0,ALZR11,0.000282,0.009940,0.219974,18.534088,0.157800,0.028413,0.501019,-0.439493
1,BBAS3,0.000819,0.024902,-0.173227,9.356940,0.395303,0.032908,0.508475,-0.582870
2,BBSE3,0.000521,0.017734,-0.049028,4.314499,0.281514,0.029393,0.515475,-0.440436
3,BRAV3,0.000132,0.030755,0.271719,2.809773,0.488217,0.004280,0.481275,-0.732606
4,GGRC11,0.000158,0.010516,-1.675901,25.424217,0.166939,0.014999,0.502222,-0.377483


## 1. Análise exploratória dos preços

- Volatilidade
- Retornos
- Distribuições
- Anomalias
- Comportamento cross-sectional

## 1.1 Cálculo de retornos e volatilidade (EDA)

In [44]:
# %% 1.1 Cálculo de retornos e volatilidade (EDA)

# Retorno simples e log-retorno (1 dia)
df["ret_close_1d"] = df.groupby(TICKER_COL)[PRICE_COL].pct_change()
df["log_ret_close_1d"] = np.log1p(df["ret_close_1d"])  # log(1 + r)

# Volatilidade rolling (21 dias, baseada em log-retornos) anualizada
window_vol = 21
df["vol_21d_eda"] = (
    df.groupby(TICKER_COL)["log_ret_close_1d"]
      .rolling(window_vol, min_periods=10)
      .std()
      .reset_index(level=0, drop=True) * np.sqrt(252)
)

# Estatísticas de preço por ativo
price_stats = (
    df.groupby(TICKER_COL)[PRICE_COL]
      .agg(["min", "max", "mean", "median", "std"])
      .rename(columns={"std": "std_price"})
)

# Estatísticas de retorno por ativo (usando funções nomeadas)
ret_stats = (
    df.groupby(TICKER_COL)["ret_close_1d"]
      .agg(
          mean_ret="mean",
          median_ret="median",
          std_ret="std",
          skew_ret="skew",
          kurt_ret=lambda x: x.kurt()
      )
)

# Estatísticas de volatilidade por ativo
vol_stats = (
    df.groupby(TICKER_COL)["vol_21d_eda"]
      .agg(
          mean_vol21="mean",
          median_vol21="median",
          max_vol21="max"
      )
)

price_ret_summary = price_stats.join(ret_stats).join(vol_stats)

print("Resumo de preço/retorno/volatilidade por ativo (head):")
display(price_ret_summary.head(10))

Resumo de preço/retorno/volatilidade por ativo (head):


,min,max,mean,median,std_price,mean_ret,median_ret,std_ret,skew_ret,kurt_ret,mean_vol21,median_vol21,max_vol21
ticker,,,,,,,,,,,,,
ALZR11,5.554731,10.655980,8.214996,8.408793,1.187229,0.000282,0.000077,0.009940,0.219974,18.534088,0.130620,0.111095,0.703060
BBAS3,3.433935,29.217079,13.611234,11.936139,6.478086,0.000819,0.000452,0.024902,-0.173227,9.356940,0.347837,0.312532,1.604658
BBSE3,9.302742,40.418262,19.483760,16.448271,6.992751,0.000521,0.000599,0.017734,-0.049028,4.314499,0.257488,0.237275,0.986526
BRAV3,13.720000,51.310001,30.974645,31.850000,8.700341,0.000132,-0.000638,0.030755,0.271719,2.809773,0.459246,0.428818,0.925266
GGRC11,6.400176,10.287263,8.807023,8.973860,0.774809,0.000158,0.000083,0.010516,-1.675901,25.424217,0.143145,0.128228,0.803092
HGRE11,67.284492,154.920868,101.220083,101.093491,11.766717,0.000186,0.000000,0.012985,-0.620200,19.557124,0.181287,0.164708,1.097117
HGRU11,65.046928,127.449997,93.856664,90.371639,14.964814,0.000326,0.000000,0.010503,-1.535653,42.599607,0.139946,0.120374,0.866627
HSML11,46.445808,88.000000,68.660819,67.012436,10.482889,0.000210,0.000000,0.011883,-1.465864,32.879009,0.146998,0.122239,1.067915
IRDM11,50.419067,96.305763,65.196109,63.257622,8.289695,-0.000090,0.000102,0.010047,-1.122455,10.193869,0.145642,0.131226,0.440613
